# Part 2: Datasets/DataFrames

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import ChiSqSelector, RegexTokenizer, CountVectorizer, StringIndexer, StopWordsRemover
from pyspark.sql import SparkSession

In [2]:
input_path = "hdfs:///user/dic23_shared/amazon-reviews/full/reviews_devset.json"
# input_path = "hdfs:///user/e11809642/reviews/reduced_devset.json"

Create Spark Session  

In [3]:
spark = SparkSession.builder.appName("ChiSquaredPipeline").getOrCreate()
sc = spark.sparkContext

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
23/05/24 14:37:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/24 14:37:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/24 14:37:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/24 14:37:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/05/24 14:37:48 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/05/24 14:37:5

Read dataset into DataFrame

In [4]:
df = spark.read.json(input_path)

tokenize

In [5]:
pattern = "[^a-zA-Z<>^|]+"
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern=pattern, gaps=True)
df = tokenizer.transform(df)

stopword removal (using local `stopwords.txt`)

In [6]:
stopwords = sc.textFile("file:///./stopwords.txt").collect()
remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=stopwords)
df = remover.transform(df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=e11810909, access=EXECUTE, inode="/user/e11809642":e11809642:supergroup:drwx------
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkTraverse(FSPermissionChecker.java:422)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:333)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermissionWithContext(FSPermissionChecker.java:370)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:240)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkTraverse(FSPermissionChecker.java:713)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkTraverse(FSDirectory.java

count term frequencies

In [ ]:
# Convert the category column to a numeric type
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
df = indexer.fit(df).transform(df)

# Compute the term frequency for each term in each category
cv = CountVectorizer(inputCol="filtered", outputCol="features", binary=True)
cv_model = cv.fit(df)
df = cv_model.transform(df)

feature selection w/ Chi Square (top 2000 features)

In [ ]:
css = ChiSqSelector(featuresCol="features", outputCol="selectedFeatures", labelCol="categoryIndex", numTopFeatures=2000)
css_model = css.fit(df)
df = css_model.transform(df)

In [ ]:
selected_features = css_model.selectedFeatures

feature_names = [cv_model.vocabulary[i] for i in selected_features]

Create pipeline combining all steps

In [ ]:
pipeline = Pipeline(stages=[tokenizer, remover, indexer, cv, css])

In [ ]:
spark.stop()